In [2]:
from torch import empty , cat , arange, Tensor
from torch.nn.functional import fold, unfold
import torch

class Module(object):
    def __init__(self):
        super().__init__()

    def forward(self, *input):
        raise NotImplementedError
    
    def backward(self, *gradwrtoutput):
        raise NotImplementedError

    def param(self):
        return []
        
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.padding = padding

        if (isinstance(kernel_size, int)):
            self.kernel_size = (kernel_size, kernel_size)
        elif (isinstance(kernel_size, tuple)):
            self.kernel_size = kernel_size

        if (isinstance(stride, int)):
            self.stride = (stride, stride)
        elif (isinstance(stride, tuple)):
            self.stride = stride    

        self.weight = empty((in_channels, out_channels, self.kernel_size[0], self.kernel_size[1]))
        self.bias = empty(out_channels)

        self.weightGrads = empty((in_channels, out_channels, self.kernel_size[0], self.kernel_size[1])).zero_()
        self.biasGrads = empty(out_channels).zero_()
        self.cache = {}

In [179]:
inp = torch.randn((50, 3, 32, 32))
gradwrtoutput = torch.randn((50, 10, 30, 30))
w = torch.randn((3, 10, 3, 3))

In [176]:
w.reshape(10, -1).shape

torch.Size([10, 27])

In [180]:
gradwrtoutput.permute(1, 2, 3, 0).reshape(10, -1).shape

torch.Size([10, 45000])

In [181]:
torch.nn.functional.unfold(inp, 3, 1).shape

torch.Size([50, 27, 900])

In [166]:
a = gradient.permute(1, 2, 3, 0).reshape(10,-1) @ torch.nn.functional.unfold(inp, 3, 1).permute(2, 0, 1).reshape(45000, -1)

In [170]:
gradient.permute(1, 2, 3, 0).reshape(10,-1).shape

torch.Size([10, 45000])

In [172]:
b= w.reshape(10,-1).t() @ gradient.permute(1, 2, 3, 0).reshape(10,-1)
b.shape

torch.Size([27, 45000])

In [175]:
a.reshape((3,10,3,3)).reshape(torch.nn.functional.unfold(inp, 3, 1).permute(1, 2, 0).shape)

RuntimeError: shape '[27, 900, 50]' is invalid for input of size 270

In [ ]:
x #bs, ic, H, W

In [71]:
w = w.view(10,-1)
w.shape

torch.Size([10, 27])

In [102]:
bias = torch.arange(10)
bias

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [113]:
bias.view(-1,1)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])

In [70]:
unfold = torch.nn.functional.unfold(inp, (3,3), (1,1))
unfold.size()

torch.Size([50, 27, 900])

In [114]:
((w @ unfold) +bias.view(-1,1)).view(50,10,30,-1).shape

torch.Size([50, 10, 30, 30])

In [115]:
().fold((30,30),(3,3)).shape

AttributeError: 'Tensor' object has no attribute 'fold'

In [54]:
a=unfold.reshape(-1,27).mm(w.reshape(10,-1).t()).reshape(50,900,-1).transpose(1, 2)
a.size()

torch.Size([50, 10, 900])

In [119]:
((w @ unfold) + bias.view(1,-1,1)).shape

torch.Size([50, 10, 900])

In [121]:
torch.nn.functional.fold(((w @ unfold) + bias.view(-1,1)), (30,30), (3,3))

RuntimeError: Expected size of input's dimension 1 to be divisible by the product of kernel_size, but got input.size(1)=10 and kernel_size=(3, 3).